Prepare Linux Environment & Import Packages

In [0]:
#!kill -9 -1
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/netflix_data
!cp /content/drive/My\ Drive/Colab_Notebooks/netflix-prize-data.zip /content/netflix_data/netflix-prize-data.zip 
%cd /content/netflix_data/
!unzip netflix-prize-data.zip
!pip install surprise

import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('darkgrid')

import os
import sys
import pickle
import random
import matplotlib
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from surprise import Reader, Dataset
from surprise import BaselineOnly
from surprise import KNNBaseline
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/netflix_data
Archive:  netflix-prize-data.zip
  inflating: README                  
  inflating: combined_data_1.txt     
  inflating: combined_data_2.txt     
  inflating: combined_data_3.txt     
  inflating: combined_data_4.txt     
  inflating: movie_titles.csv        
  inflating: probe.txt               
  inflating: qualifying.txt          
     |████████████████████████████████| 6.5MB 3.1MB/s 
  Created wheel for scikit-surprise: filename=scik

### 1. Reading and Storing Data

#### 1.1 Data Pre-Processing

In [0]:
if os.path.isfile('netflix_ratings.csv') is False:
    meta = open('netflix_ratings.csv', mode='w')
    files = ['combined_data_1.txt', 'combined_data_2.txt', 
            'combined_data_3.txt', 'combined_data_4.txt']
    for file in files:
        print('Reading from file {}...'.format(str(file)))
        with open(file) as f:
            # as long as the file is open, perform the underneath operations
            for line in f:
                line = line.strip() # clears all the leading and trailing spaces from the string
                if line.endswith(':'):
                    movieID = line.replace(':', '')
                else:
                    row = []
                    row = [value for value in line.split(',')]
                    row.insert(0, movieID)
                    meta.write(','.join(row))
                    meta.write('\n')
        print('Successfully completed reading the file {}'.format(str(file)))
    meta.close()

Reading from file combined_data_1.txt...
Successfully completed reading the file combined_data_1.txt
Reading from file combined_data_2.txt...
Successfully completed reading the file combined_data_2.txt
Reading from file combined_data_3.txt...
Successfully completed reading the file combined_data_3.txt
Reading from file combined_data_4.txt...
Successfully completed reading the file combined_data_4.txt


In [0]:
def save_as_pickled_object(obj, filepath):
    """
    This is a defensive way to write pickle.write, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    bytes_out = pickle.dumps(obj)
    n_bytes = sys.getsizeof(bytes_out)
    with open(filepath, 'wb') as f_out:
        for idx in range(0, n_bytes, max_bytes):
            f_out.write(bytes_out[idx:idx+max_bytes])


def try_to_load_as_pickled_object_or_None(filepath):
    """
    This is a defensive way to write pickle.load, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    try:
        input_size = os.path.getsize(filepath)
        bytes_in = bytearray(0)
        with open(filepath, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        obj = pickle.loads(bytes_in)
    except:
        return None
    return obj

if os.path.isfile('netflix_meta.pkl') is False:
    meta = pd.read_csv('netflix_ratings.csv', sep=',',
                       names=['movie_id', 'customer_id', 'rating', 'date'])
    meta['date'] = pd.to_datetime(meta['date'])
    meta.sort_values(by='date', inplace=True)
    save_as_pickled_object(meta, 'netflix_meta.pkl')
    print('Successfully transformed netflix_ratings.csv to netflix_meta.pkl.')

meta = try_to_load_as_pickled_object_or_None('netflix_meta.pkl')
print('Successfully loaded netflix_meta.pkl.')

Successfully transformed netflix_ratings.csv to netflix_meta.pkl.
Successfully loaded netflix_meta.pkl.


In [0]:
meta.head()

,movie_id,customer_id,rating,date
56431994,10341,510180,4,1999-11-11
9056171,1798,510180,5,1999-11-11
58698779,10774,510180,3,1999-11-11
48101611,8651,510180,2,1999-11-11
81893208,14660,510180,2,1999-11-11


#### 1.2 Data Cleaning

##### 1.2.1 Checking NaN

In [0]:
print('The number of NaN values: \n{}'.format(meta.isnull().sum()))

The number of NaN values: 
movie_id       0
customer_id    0
rating         0
date           0
dtype: int64


##### 1.2.2 Checking Duplications

In [0]:
duplications = meta.duplicated(['movie_id', 'customer_id', 'date'])
print('The number of duplications: \n{}'.format(duplications.sum())) 
# the function dataframe.duplicated return a series: [index, true/false]

The number of duplications: 
0


#### 1.3 Basic Statistics

In [0]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100480507 entries, 56431994 to 25464092
Data columns (total 4 columns):
movie_id       int64
customer_id    int64
rating         int64
date           datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 3.7 GB


In [0]:
meta.describe()['rating']

count    1.004805e+08
mean     3.604290e+00
std      1.085219e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [0]:
print('The summary of the meta data:',
     '\n Total number of unique users: {} \n Total number of unique movies: {} \n Total count of rating behaviors: {}'
     .format(str(len(np.unique(meta['customer_id']))), str(len(np.unique(meta['movie_id']))), str(meta.shape[0])))

The summary of the meta data: 
 Total number of unique users: 480189 
 Total number of unique movies: 17770 
 Total count of rating behaviors: 100480507


### 2. Split Validation: Splitting Data into Train Data and Test Data

In [0]:
if os.path.isfile('train_data.pkl') is False:
    train_df = meta.iloc[:int(meta.shape[0] * 0.80)]
    save_as_pickled_object(train_df, 'train_data.pkl')
train_data = try_to_load_as_pickled_object_or_None('train_data.pkl')
train_data.reset_index(drop=True, inplace=True)
print('Successfully loaded the train data after resetting the index.')

if os.path.isfile('test_data.pkl') is False:
    test_df = meta.iloc[int(meta.shape[0] * 0.80):]
    save_as_pickled_object(test_df, 'test_data.pkl')
test_data = try_to_load_as_pickled_object_or_None('test_data.pkl')
test_data.reset_index(drop=True, inplace=True)
# Otherwise, the index will start from 32637772, 55518921, 96207802...
print('Successfully loaded the test data after resetting the index.')

Successfully loaded the train data after resetting the index.
Successfully loaded the test data after resetting the index.


In [0]:
train_data.head()

,movie_id,customer_id,rating,date
0,10341,510180,4,1999-11-11
1,1798,510180,5,1999-11-11
2,10774,510180,3,1999-11-11
3,8651,510180,2,1999-11-11
4,14660,510180,2,1999-11-11


In [0]:
print('The summary of the train data:',
     '\n Total number of unique users: {} \n Total number of unique movies: {} \n Total count of rating behaviors: {}'
     .format(str(len(np.unique(train_data['customer_id']))), str(len(np.unique(train_data['movie_id']))), str(train_data.shape[0])))

The summary of the train data: 
 Total number of unique users: 405041 
 Total number of unique movies: 17424 
 Total count of rating behaviors: 80384405


In [0]:
test_data.head()

,movie_id,customer_id,rating,date
0,5926,2294429,2,2005-08-08
1,10158,1743373,4,2005-08-08
2,17064,381625,5,2005-08-08
3,1443,1252933,5,2005-08-08
4,1201,1434500,4,2005-08-08


In [0]:
print('The summary of the test data:',
     '\n Total number of unique users: {} \n Total number of unique movies: {} \n Total count of rating behaviors: {}'
     .format(str(len(np.unique(test_data['customer_id']))), str(len(np.unique(test_data['movie_id']))), str(test_data.shape[0])))

The summary of the test data: 
 Total number of unique users: 349312 
 Total number of unique movies: 17757 
 Total count of rating behaviors: 20096102


### 3. Matrix Factorization-based Algorithms: SVD & SVD++
We use Root Mean Squared Error (RMSE) to predict as our accuracy metric.

#### 3.1 Tuning algorithm parameters with GridSearchCV

In [0]:
if os.path.isfile('meta_surprise.pkl') is False:
    meta_excerpt = meta.sample(frac=1).reset_index(drop=True)
    meta_excerpt = meta_excerpt.iloc[:int(meta_excerpt.shape[0] * 0.30)]
    
    reader = Reader(rating_scale=(0, 5))
    meta_surprise = Dataset.load_from_df(meta_excerpt.iloc[:,:3], reader)
    
    save_as_pickled_object(meta_surprise, 'meta_surprise.pkl')
    print('Successfully constructed meta_surprise from Dataframe meta as the original dataset format in Surprise.')

meta_surprise = try_to_load_as_pickled_object_or_None('meta_surprise.pkl')
print('Successfully loaded meta_surprise.pkl.')

NameError: ignored

In [0]:
algorithms_list = [SVD(), SVDpp()]

best_rmse_list = []
best_param_list = []
index_list = []

for algorithm in algorithms_list:
    # e.g. '<surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x1012bf0b8>'
    index_name = str(algorithm).split(' ')[0].split('.')[-1]
    index_list.append(index_name)
    
    print('Start to optimize the parameters of the algorithm {}. \n'.format(index_name))
    
#     param_grid = {'n_factors': [50,100,150],
#                   'n_epochs': [20,30], 
#                   'lr_all': [0.005,0.01]}

    param_grid = {'n_factors': [100], 
                  'n_epochs': [20], 
                  'lr_all': [0.005], 
                  'reg_all': [0.02]} # Default hyperparameters
    
    gs = GridSearchCV(algorithm, param_grid, measures=['rmse'], cv=3)
    gs.fit(meta_surprise)
    
    best_rmse = gs.best_score['rmse']
    best_param = gs.best_params['rmse']
    
    best_rmse_list.append(best_rmse)
    best_param_list.append(best_param)
    
    print('Successfully finished the optimization of the algorithm {}. \n'.format(index_name))

Start to optimize the parameters of the algorithm SVD. 



In [0]:
evaluation_dict = {}
evaluation_dict.update({'Algorithm': algorithms_list})
evaluation_dict.update({'Best RMSE': best_rmse_list})
evaluation_dict.update({'Best Parameter': best_param_list})
    
evaluation_df = pd.DataFrame(evaluation_dict).sort_values('Best RMSE') 
print('Successfully constructed the evaluation dataframe.')

evaluation_df.to_csv('evaluation_df.csv')
print('Successfully constructed the evaluation dataframe as a csv file.')

#### 3.2 Applying the algorithm with the best parameters